In [5]:
import numpy as np
import os

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import time

In [3]:
#utility function to make this notebook's output stable across the runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

CREATING AND RUNNING A GRAPH

In [2]:
import tensorflow as tf

C:\Users\Priyam\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
#creating variables on the default graph
reset_graph()

#Variables and constants 
x=tf.Variable(3, name='x') #variable x initialised to value=3
y=tf.Variable(4, name='y') #variable y initialised to value=4
a=tf.constant(2, name='a')

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
print(x.graph is tf.get_default_graph())
print(y.graph is tf.get_default_graph())
print(a.graph is tf.get_default_graph())

#This shows all the variables and constants are created on the default graph

True
True
True


In [11]:
#Nodes can also be created on a specific graph
graph=tf.Graph()

with graph.as_default():
    x2=tf.Variable(2)
    
print(x2.graph is graph)
print(x2.graph is tf.get_default_graph()) 
#This shows that x2 variable is not created in the default graph but in a specific graph

True
False


In [12]:
x,y,a
#NOTE: the variables and constants are not yet initialised

(<tf.Variable 'x:0' shape=() dtype=int32_ref>,
 <tf.Variable 'y:0' shape=() dtype=int32_ref>,
 <tf.Tensor 'a:0' shape=() dtype=int32>)

In [13]:
#Define Function
f=x*x*y+y+a

In [14]:
f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

__To Evaluate the Graph we open a TensorFlow session__<br>
A tf session initializes all the variables and evaluates the graph. 
It puts the graph on a CPU or GPU or cluster and holds all the variables values

In [16]:
#A verbose way
sess=tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result=sess.run(f)
a_val=a.eval(session=sess) #a constant needs not be initiated
print("a=",a_val)
print("result=",result)
sess.close()

a= 2
result= 42


In [18]:
#Convenient Way, this closes the session automatically
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    a_val=a.eval()
    result=f.eval() #same as sess.run(f)
    
print("a=",a_val)
print("result=",result)


a= 2
result= 42


__Initialization of all variables at once__

In [19]:
init=tf.global_variables_initializer()

with tf.Session() as sess:
    init.run() #initializes all the variables
    result=f.eval()
    
print(result)
    

42


In [20]:
init

<tf.Operation 'init' type=NoOp>

__Lifecycle of a node value__

In [30]:
w=tf.constant(3)
x1=w+2
y1=x1+5
z1=x1+3

In [27]:
with tf.Session() as sess1:
    print(y1.eval())
    print(z1.eval())

10
8


Above code worked because only w(a constant) was being used and no variables were present to be initialised. 
Constants dont have to be initialised.<br> Also for calculating y1 and z1, session has to be initialised twice to traverse through the graph. In order to calculate, y1 and z1 in a single flow we should use below:

In [32]:
with tf.Session() as sess1:
    y_val,z_val=sess1.run([y1,z1])
    print(y_val)
    print(z_val)

10
8


If we were using variable instead of a constant then it would have to be initialised as shown below:

In [8]:
reset_graph()
w2=tf.Variable(82,'w2')
x1=w2+2
y1=x1+5
z1=x1+3

In [ ]:
init=tf.global_variables_initializer()
with tf.Session() as sess1:
    init.run()
    y_val,z_val=sess1.run([y1,z1])
    print(y_val)
    print(z_val)

__Reverse AutoDiff in Tensorflow__

Define a composite function: f(w)=exp($w_{20}$+$w_{21}$.exp($w_{10}$+$w_{11}$.exp($w_{00}$+$w_{01}$.x)))

In [10]:
def my_func(w,x):
    f_0=tf.exp(w[0,0]+w[0,1]*x)
    f_1=tf.exp(w[1,0]+w[1,1]*f_0)
    f_2=tf.exp(w[2,0]+w[2,1]*f_1)
    return f_2,f_1,f_0

In [32]:
#A fancier implementation using name scope
def my_func2(w,x):
    with tf.name_scope("f_0_level") as scope_0:
        f_0=tf.exp(w[0,0]+w[0,1]*x)
    with tf.name_scope("f_1_level") as scope_1:
        f_1=tf.exp(w[1,0]+w[1,1]*f_0)
    with tf.name_scope("f_2_level") as scope_2:
        f_2=tf.exp(w[2,0]+w[2,1]*f_1)
        
    return f_2,f_1,f_0

In [29]:
#Now we need to pick a point for w where all intercepts are 0 and all slopes are 1
w_0=np.vstack((np.zeros(3),np.ones(3))).T

In [27]:
#This can also be used. c can be used as w_0
a=np.zeros(3).reshape(3,1)
b=np.ones(3).reshape(3,1)
c=np.append(a,b,axis=1)

In [28]:
c

array([[0., 1.],
       [0., 1.],
       [0., 1.]])

__Computing gradients using Tensorflow__

In [35]:
reset_graph()
#Variables and their initialization
w=tf.Variable(w_0,name='w',dtype=tf.float32)
x=tf.Variable(1,name='x',dtype=tf.float32,trainable=False)

#Define nodes for values of the function at layers
f_2,f_1,f_0=my_func2(w,x)

grads=tf.gradients(f_2,w)
grads

[<tf.Tensor 'gradients/AddN:0' shape=(3, 2) dtype=float32>]

In [40]:
#A node for initializer
init=tf.global_variables_initializer()

#Run the session
t_0=time.time()

with tf.Session() as sess:
    sess.run(init)
    #function_vals=sess.run([f_2,f_1,f_0])
    #gradients=sess.run(grads)
    
    gradients, function_vals=sess.run([grads,[f_2,f_1,f_0]])
    
print("Computed the derivatives in ",(time.time()-t_0)," seconds")
print("Function values:", function_vals)
print("Gradients=",gradients)

Computed the derivatives in  0.04089021682739258  seconds
Function values: [3814273.0, 15.154261, 2.7182817]
Gradients= [array([[1.5712344e+08, 1.5712344e+08],
       [5.7802488e+07, 1.5712344e+08],
       [3.8142730e+06, 5.7802488e+07]], dtype=float32)]


_Graphs can be visualised using Notebook or Tensorboard. Below is using Notebook_